<div align="center">
  <a target="_blank" href="https://colab.research.google.com/github/crowdcent/crowdcent-challenge/blob/main/docs/tutorials/hyperliquid-example.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
  </a>
</div>

## Build a model on CrowdCent's training data and submit

### Load API key

For this tutorial, you will need:
1) **CrowdCent API Key**: register for free at https://crowdcent.com/accounts/signup/

In [1]:
%load_ext dotenv
%dotenv

import os
import crowdcent_challenge as cc
import polars as pl
from catboost import CatBoostRegressor

CROWDCENT_API_KEY = os.environ["CROWDCENT_API_KEY"]

### Initialize the client

In [ ]:
client = cc.ChallengeClient(
    challenge_slug="hyperliquid-ranking",
    api_key=CROWDCENT_API_KEY,
    base_url="http://localhost:8080/api",
)

### Get CrowdCent's training data

In [ ]:
client.download_training_data(
    version="latest", dest_path="training_data.parquet"
)

training_data = pl.read_parquet("training_data.parquet")
training_data.head()

### Train a model on the training data

In [ ]:
cb_regressor = CatBoostRegressor(loss_function="MultiRMSE")
feature_names = [col for col in training_data.columns if col.startswith("feature")]

cb_regressor.fit(
    training_data[feature_names],
    training_data[["target_10d", "target_30d"]],
)

### Get CrowdCent's latest inference data

In [ ]:
# inference tickers
client.download_inference_data(
    release_date="current", dest_path="inference_data.parquet"
)

inference_data = pl.read_parquet("inference_data.parquet")
inference_data.head()

### Make predictions on the inference data

In [ ]:
preds = cb_regressor.predict(inference_data[feature_names])
preds.head()

### Submit to the `hyperliquid-ranking` challenge on CrowdCent

In [ ]:
client.submit_predictions(preds)